In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mnist_example import MNISTDataModule, MNISTModel

datamodule = MNISTDataModule(batch_size=10)
datamodule.setup()

len(datamodule.test_dataloader())

10

In [3]:
from energizer.data.datamodule import ActiveDataModule

dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

dm.is_on_pool = False
print(len(dm.test_dataloader()))
dm.is_on_pool = True
print(len(dm.test_dataloader()))

10
10


In [4]:
# print(
#     len(dm.pool_fold),
#     len(dm.test_dataloader()),
#     len(dm.train_fold),
#     len(dm.train_dataloader()),
# )
# dm.label(list(range(100)))
# print(
#     len(dm.pool_fold),
#     len(dm.test_dataloader()),
#     len(dm.train_fold),
#     len(dm.train_dataloader()),
# )

---

In [5]:
import torch
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer as pl_Trainer
from torch.utils.data import DataLoader

from energizer.learners.acquisition_functions import entropy
from energizer.learners.base import Deterministic, Learner
from energizer.trainer import Trainer

In [6]:
class ActiveLearner(Deterministic):
    def pool_step(self, batch, *args, **kwargs) -> torch.Tensor:
        x, _ = batch
        logits = self(x)
        scores = entropy(logits)
        return scores


model = MNISTModel()
active_learner = ActiveLearner(model)

In [43]:
trainer = Trainer(
    query_size=2,
    max_epochs=3,
    max_labelling_epochs=2,
    log_every_n_steps=1,
    test_after_labelling=True,
)
trainer.active_fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 51.56it/s, loss=1.33, v_num=9]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 57.31it/s, loss=1.14, v_num=9]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 38.04it/s, loss=1.14, v_num=9]
